In [8]:
# Dependencies
import modules.dataset.entities as en
import modules.dataset.tweets as tw
import matplotlib.pyplot as plt

%matplotlib inline

# Constants
years = [2018, 2019]

## Dataset creation

Once words and hashtags are extracted, they are stored in Pandas DataFrames with the following attributes:

1. <code> id </code> : Reference to the tweet ID
2. <code> index </code> : Position of the word in the sentence (tweet) - useful to delete self loops (WORDS ONLY)
3. <code> text </code> : Text
4. <code> pos </code> : Part Of Speech tag (WORDS ONLY)
5. <code> conf </code> : Confidence associated to the POS tag (WORDS ONLY)